In [2]:
from starter import *
import pickle

%matplotlib inline

In [3]:
from datetime import datetime, date, time

## Load data

In [4]:
train = pd.read_csv('train.csv')
holidays = pd.read_csv('holidays.csv')
weather = pd.read_csv('weather.csv')
metadata = pd.read_csv('metadata.csv')
submission_format = pd.read_csv('submission_format.csv')
submission_frequency = pd.read_csv('submission_frequency.csv')

In [5]:
data = {'train': train, 'holidays': holidays, 'weather': weather, 'metadata': metadata,
       'submission_format': submission_format, 'submission_frequency': submission_frequency}

## Data Cleaning

### Fix timestamps

In [26]:
train['Timestamp'] = pd.to_datetime(train.Timestamp)
holidays['Date'] = pd.to_datetime(holidays.Date).dt.date
weather['Timestamp'] = pd.to_datetime(weather.Timestamp)
submission_format['Timestamp'] = pd.to_datetime(submission_format.Timestamp)

In [60]:
train['Date'] = train['Timestamp'].dt.date
train['Year'] = train['Timestamp'].dt.year
train['Month'] = train['Timestamp'].dt.month
train['Time'] = train['Timestamp'].dt.time
train['Hour'] = train['Timestamp'].dt.hour

In [19]:
submission_format['Date'] = submission_format['Timestamp'].dt.date
submission_format['Year'] = submission_format['Timestamp'].dt.year
submission_format['Month'] = submission_format['Timestamp'].dt.month
submission_format['Time'] = submission_format['Timestamp'].dt.time
submission_format['Hour'] = submission_format['Timestamp'].dt.hour

In [61]:
weather['Time'] = weather['Timestamp'].dt.time

In [78]:
submission_frequency.loc[:,'ForecastPeriodMin'] = submission_frequency.ForecastPeriodNS/(6e10)

In [79]:
submission_frequency.loc[:,'Period_Quarter'] = submission_frequency.apply(
    lambda row: 1 if row['ForecastPeriodMin']==15 else 0, axis=1)
submission_frequency.loc[:,'Period_Hour'] = submission_frequency.apply(
    lambda row: 1 if row['ForecastPeriodMin']==60 else 0, axis=1)
submission_frequency.loc[:,'Period_Days'] = submission_frequency.apply(
    lambda row: 1 if row['ForecastPeriodMin']==1440 else 0, axis=1)

In [80]:
submission_frequency.head()

,ForecastId,ForecastPeriodNS,ForecastPeriodMin,Period_Quarter,Period_Hour,Period_Days
0,1,86400000000000,1440.0,0,0,1
1,2,86400000000000,1440.0,0,0,1
2,3,86400000000000,1440.0,0,0,1
3,4,86400000000000,1440.0,0,0,1
4,5,3600000000000,60.0,0,1,0


In [10]:
# pickle.dump(train, open('train.p', 'wb'))
# pickle.dump(weather, open('weather.p', 'wb'))
# pickle.dump(holidays, open('holidays.p', 'wb'))
# pickle.dump(submission_frequency, open('submission_frequency.p', 'wb'))
train = pickle.load(open('train.p', 'rb'))
weather = pickle.load(open('weather.p', 'rb'))
holidays = pickle.load(open('holidays.p', 'rb'))
submission_frequency = pickle.load(open('submission_frequency.p', 'rb'))

### Fix missing and non-numeric values

In [62]:
#replace missing values with 0
train.Value.fillna(0, inplace=True)

In [31]:
#replace weather temperature with average of each time
weather_grouped = weather.groupby(by=['SiteId', 'Timestamp']).mean()
weather_grouped['Timestamp'] = weather_grouped.apply(
    lambda row: row.name[1], axis=1)
weather_grouped['SiteId'] = weather_grouped.apply(
    lambda row: row.name[0], axis=1)

In [34]:
weather_grouped['Timestamp'] = pd.to_datetime(weather_grouped['Timestamp'])
weather_grouped.index = weather_grouped['Unnamed: 0']
weather_grouped.drop('Unnamed: 0', axis=1, inplace=True)
weather_grouped.reset_index(inplace=True)

In [8]:
#replace True and False with 1 and 0
for feature in range(4,11):
    metadata[metadata.columns[feature]].replace({True:1, False:0}, inplace=True)
metadata.index = metadata.SiteId

### Feature Engineering

Add day of week

In [21]:
train['DayofWeek'] = pd.to_datetime(train['Date']).dt.weekday_name
submission_format['DayofWeek'] = pd.to_datetime(submission_format['Date']).dt.weekday_name

Change holidays to 1

In [27]:
holidays['isHoliday'] = 1

Add temperature to metadata

In [13]:
# finds sites that do not have sufficient temperature data
def no_temp(data):
    delete_list = []
    for site in data.SiteId.unique():
        threshold_size = data[data.SiteId==site].shape[0]*0.5
        try:
            if data[data.SiteId==site].Temperature.isnull().value_counts()[0] < threshold_size:
                delete_list.append(site)
        except KeyError:
            delete_list.append(site)
    return delete_list

In [14]:
metadata['Temperature'] = 1
metadata.loc[no_temp(df),'Temperature'] = 0

In [6]:
# pickle.dump(train, open('train.p', 'wb'))
# pickle.dump(weather, open('weather.p', 'wb'))
# pickle.dump(metadata, open('metadata.p', 'wb'))
# pickle.dump(weather_grouped, open('weather_grouped.p','wb'))
# pickle.dump(holidays, open('holidays.p', 'wb'))
# pickle.dump(submission_format, open('submission_format.p', 'wb'))
train = pickle.load(open('train.p', 'rb'))
weather = pickle.load(open('weather.p', 'rb'))
metadata = pickle.load(open('metadata.p', 'rb'))
weather_grouped = pickle.load(open('weather_grouped.p', 'rb'))
holidays = pickle.load(open('holidays.p', 'rb'))
submission_format = pickle.load(open('submission_format.p', 'rb'))

## Formatting for submission

In [7]:
submission_format.head()

,obs_id,SiteId,Timestamp,ForecastId,Value,Date,Year,Month,Time,Hour,DayofWeek
0,1677832,1,2015-08-29,1,0.0,2015-08-29,2015,8,00:00:00,0,Saturday
1,5379616,1,2015-08-30,1,0.0,2015-08-30,2015,8,00:00:00,0,Sunday
2,496261,1,2015-08-31,1,0.0,2015-08-31,2015,8,00:00:00,0,Monday
3,4567147,1,2015-09-01,1,0.0,2015-09-01,2015,9,00:00:00,0,Tuesday
4,3684873,1,2015-09-02,1,0.0,2015-09-02,2015,9,00:00:00,0,Wednesday


In [8]:
train.head()

,obs_id,SiteId,Timestamp,Value,Date,Time,Hour,DayofWeek,Year,Month
0,744519,1,2014-09-03,9.096555e+05,2014-09-03,00:00:00,0,Wednesday,2014,9
1,7627564,1,2014-09-04,1.748273e+06,2014-09-04,00:00:00,0,Thursday,2014,9
2,7034705,1,2014-09-05,0.000000e+00,2014-09-05,00:00:00,0,Friday,2014,9
3,5995486,1,2014-09-06,0.000000e+00,2014-09-06,00:00:00,0,Saturday,2014,9
4,7326510,1,2014-09-07,0.000000e+00,2014-09-07,00:00:00,0,Sunday,2014,9


## Prepare dataframe

In [71]:
# merge weather data
df = train.merge(weather_grouped, how='left', on=['Timestamp','SiteId'])
# merge holidays data
df = df.merge(holidays, how='left', on=['Date','SiteId'])

This killed the kernel...

In [ ]:
df.merge(metadata)

In [10]:
metadata.head()

,SiteId,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff
0,1,1387.205119,15.0,18.0,0,0,0,0,0,1,1
1,2,6098.278376,30.0,18.0,0,0,0,0,0,1,1
2,3,10556.293605,5.0,18.0,0,0,0,0,0,1,0
3,5,12541.181277,30.0,18.0,0,0,0,0,0,1,1
4,6,9150.195373,30.0,18.0,0,0,0,0,0,1,1


Use this to filter out all the times that we do not need

In [132]:
df[(df.SiteId==1) | (df.SiteId==2)].head()

,obs_id,SiteId,Timestamp,Value,Date,Time,Hour,Unnamed: 0,Temperature,Distance,Year
0,744519,1,2014-09-03,9.096555e+05,2014-09-03,00:00:00,0,90323.5,19.60,22.921092,2014.0
1,7627564,1,2014-09-04,1.748273e+06,2014-09-04,00:00:00,0,90349.0,21.30,22.921092,2014.0
2,7034705,1,2014-09-05,0.000000e+00,2014-09-05,00:00:00,0,90375.0,23.35,22.921092,2014.0
3,5995486,1,2014-09-06,0.000000e+00,2014-09-06,00:00:00,0,90399.5,21.60,22.921092,2014.0
4,7326510,1,2014-09-07,0.000000e+00,2014-09-07,00:00:00,0,90425.0,15.80,22.921092,2014.0


In [12]:
# pickle.dump(df, open('df.p', 'wb'))
df = pickle.load(open('df.p', 'rb'))

In [127]:
df[df.SiteId==1]

,obs_id,SiteId,Timestamp,Value,Date,Time,Hour,Unnamed: 0,Temperature,Distance,Year
0,744519,1,2014-09-03,9.096555e+05,2014-09-03,00:00:00,0,90323.5,19.60,22.921092,2014.0
1,7627564,1,2014-09-04,1.748273e+06,2014-09-04,00:00:00,0,90349.0,21.30,22.921092,2014.0
2,7034705,1,2014-09-05,0.000000e+00,2014-09-05,00:00:00,0,90375.0,23.35,22.921092,2014.0
3,5995486,1,2014-09-06,0.000000e+00,2014-09-06,00:00:00,0,90399.5,21.60,22.921092,2014.0
4,7326510,1,2014-09-07,0.000000e+00,2014-09-07,00:00:00,0,90425.0,15.80,22.921092,2014.0
5,3625268,1,2014-09-08,1.964878e+06,2014-09-08,00:00:00,0,90451.0,15.25,22.921092,2014.0
6,6038546,1,2014-09-09,3.266904e+06,2014-09-09,00:00:00,0,90477.0,19.15,22.921092,2014.0
7,4114748,1,2014-09-10,2.926094e+06,2014-09-10,00:00:00,0,90503.0,19.25,22.921092,2014.0
8,2572804,1,2014-09-11,1.720502e+06,2014-09-11,00:00:00,0,90529.0,23.55,22.921092,2014.0
9,2078763,1,2014-09-12,1.136248e+06,2014-09-12,00:00:00,0,90555.0,12.05,22.921092,2014.0


In [70]:
df[df.Temperature.isna()].SiteId.value_counts(ascending=False)

302    167975
261    156168
271    125318
269    118844
276    106040
190    102184
232     92544
14      92544
22      90603
298     90499
121     84701
123     84701
287     84364
122     82768
6       75692
33      72804
115     71950
221     67204
280     63624
120     63447
295     62545
25      62386
119     62062
8       61572
87      61306
237     60976
140     60277
200     58086
141     57840
41      57837
        ...  
32         22
189        21
172        19
193        17
213        16
206        10
125         9
293         8
219         8
211         8
102         8
202         7
51          7
10          7
3           6
301         5
303         5
1           5
290         5
38          4
196         4
304         4
29          3
12          2
240         2
7           2
184         2
278         1
236         1
241         1
Name: SiteId, Length: 266, dtype: int64

In [67]:
metadata

,SiteId,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff
0,1,1387.205119,15.0,18.0,False,False,False,False,False,True,True
1,2,6098.278376,30.0,18.0,False,False,False,False,False,True,True
2,3,10556.293605,5.0,18.0,False,False,False,False,False,True,False
3,5,12541.181277,30.0,18.0,False,False,False,False,False,True,True
4,6,9150.195373,30.0,18.0,False,False,False,False,False,True,True
5,7,15168.125971,30.0,18.0,False,False,False,False,False,True,True
6,8,22221.851847,30.0,18.0,False,False,False,False,False,True,True
7,9,14588.849015,30.0,18.0,False,False,False,False,False,True,True
8,10,6393.671251,30.0,18.0,False,False,False,False,False,True,True
9,11,2517.739425,30.0,18.0,False,False,False,False,False,True,True


http://pandas.pydata.org/pandas-docs/version/0.16.2/generated/pandas.DataFrame.interpolate.html